<h1> Maximum water depth and speed in DFlowFM </h1>

<big>
<ul>
<li>
    <b>Settings are read from the <tt>*.fou</tt> file, e.g:</b><br>
    <tt>
        wl    <font color="#008800">182</font> <font color="#0000ff">184</font>    0    1.000       0.0  max<br>
uc    182 184    0    1.000       0.0  <font color="#ff0000"><b>1</b></font> max<br><br>        
    </tt><br>
    Explanation of the columns:<br>
    <ul>
        <li> variable </li>
        <li> start time, in user specified time units </li>
        <li> stop time, in user specified time units </li>
        <li><font color="#888888"> mode, only relevant for Fourier analysis </font></li>
        <li><font color="#888888"> correction parameter, only relevant for Fourier analysis </font></li>
        <li> [layer number, for some 3D variables, will be removed later]</li>
        <li> operator, can be <b>min, max or avg</b> (averaging can also be calculated as fourier mode 0)</li>
    </ul>
    <br>
    
    
</li>
<li>
    <b>Referenced from the <tt>*.mdu</tt> file:</b><br>
<tt>
[time]
...<br>
DtUser                            = 120. </tt><b>(User timestep in seconds!)</b><tt><br>
Tunit                             = H <br>
TStart                            = <font color="#008800">182.</font> </tt><b>(This is in the user specified time unit!)</b><tt><br>
TStop                             = <font color="#0000ff">184.</font> <br><br>
[output]<br>
FouFile                     = maxima.fou <br>
...<br><br>
    </tt>
</li>    
<li>
<b>Supported quantities:</b>
    <ul>
        <li> Water level (<tt>var=wl</tt>)</li>
        <li> Water depth (comes automatically with the water level)</li>
        <li> Cell-centred speed (=velocity magnitude) (<tt>var=uc</tt>)</li>
    </ul>
 </li>
</ul>
</big>

<h1> Rise rates from a DFlowFM classmap file</h1>

The rise rates are serived directly from the classmap files and written in the TIFF format<br>


In [2]:
import numpy as np
import math
import os, sys
import ugfile as uf
import tiffwriter as tw
import output_settings as c
import risingspeeds

fnin = 'case1\\bw_case1_clm.nc'
my_varname = 'mesh2d_waterdepth' 

# Open classmap file and derive rising speeds
ds = uf.DatasetUG(fnin,'r')
bbox, polygons, coords = ds.get_polygons(my_varname) 
ncvar = ds.variables[my_varname]
nctim = ds.variables["time"]
rising_speed_calc = risingspeeds.IncrementalConverter(ncvar, nctim, 0.02, 1.5)
rising_speeds = rising_speed_calc.getRisingSpeeds(verbose=True)

#bbox, polygons, coords = ds.get_polygons(my_varname) 
bbox[0] = np.floor((bbox[0]-c.lower_left[0])/c.xres)*c.xres + c.lower_left[0]
bbox[1] = np.floor((bbox[1]-c.lower_left[1])/c.yres)*c.yres + c.lower_left[1]


# Write calculated rising speeds to tiff
my_vardata = rising_speeds    
tiffname   = 'rising_speeds.tif'
nband = 1

sys.stderr.write("\n")
sys.stderr.write("Opening new TIFF:\n")
my_tiff = tw.tiffwriter(tiffname, bbox, [c.xres, c.yres], nband, c.nptype, c.epsg, flipped=True)

sys.stderr.write("Filling pixels:\n")
pixels = my_tiff.from_polygons(polygons, my_vardata, c.nodata)

sys.stderr.write("Writing Band 1 .... \n")
my_tiff.fillband(1, pixels, c.nodata)

my_tiff.close()
ds.close()


Calculating rising speed Node : 00025223,    100%
Opening new TIFF:
Filling pixels:
Writing Band 1 ....  : 00025223,    100%Writing to TIFF Node : 00001145,      5%Writing to TIFF Node : 00001493,      6%Writing to TIFF Node : 00003491,     14%Writing to TIFF Node : 00004646,     18%Writing to TIFF Node : 00008489,     34%Writing to TIFF Node : 00011488,     46%Writing to TIFF Node : 00014487,     57%Writing to TIFF Node : 00017486,     69%Writing to TIFF Node : 00019985,     79%Writing to TIFF Node : 00021984,     87%Writing to TIFF Node : 00023983,     95%
